Capstone Project - The Battle of Neighborhoods 

Part 1 Using BeautifulSoup to scrape the Toronto neighbourhoods from Wikipedia 

install and import library

In [2]:
!conda install -c ulmo beautifulsoup4      #Uncomment this if BeautifulSoup is not installed on your system
!conda install -c ulmo urllib3             #Uncomment this if urllib3 is not installed on your system
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                   failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - backports.functools_lru_cache -> python[version='>=2.7,<2.8.0a0']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that cond

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                               failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - backports.functools_lru_cache -> python[version='>=2.7,<2.8.0a0']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that conda wi

variables

In [3]:
# To scrap the data
CONST_wikipediaLink = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Key for the dataframe we will use
CONST_dfColumns = ['PostalCode', 'Borough', 'Neighborhood']

# Size of the radius to retrieve venues from FoursquareAPI, and limit of venues per neighbourhood
CONST_venuesRadiusScan = 1100
CONST_venuesLimit = 100

Using BeautifulSoup to scrape the raw data 

In [4]:
http = urllib3.PoolManager()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Scrape the raw xml data from Wikipedia
response = http.request('GET', CONST_wikipediaLink)
soup = BeautifulSoup(response.data, "lxml")

#soup

Parse the raw data

In [5]:
df_neighbourhoods = pd.DataFrame(columns=CONST_dfColumns)
fullList = []
# Retrieve all neighbourhoods, in the <tr> tags
neighbourhoodsList = soup.body.table.find_all('tr')

# Loop through each neighbourhood, in the <td> tags
for neighbourhood in neighbourhoodsList:
    neighbourElts = neighbourhood.find_all('td')
    
    # Loop through each attritube of the current neighbourhood : name, title, and wikipedia url
    tmpList = []
    for elt in neighbourElts:
        # Remove the tags & newlines
        tmpList.append(str(elt.get_text().strip()))
        
    fullList.append(tmpList)

# Add the scraped nneighbourhoods into the dataframe
df_neighbourhoods=pd.DataFrame(fullList,columns=CONST_dfColumns)
df_neighbourhoods.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Data preparation

In [6]:
# Filter all unassigned neighbourhoods
df_neighbourhoods = df_neighbourhoods[(df_neighbourhoods.Borough.notnull())]
df_neighbourhoods = df_neighbourhoods[(df_neighbourhoods.Borough != "Not assigned")]

# Group by PostalCode/Borough
df_neighbourhoods = pd.DataFrame(df_neighbourhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)).reset_index()
df_neighbourhoods['Neighborhood'] = df_neighbourhoods['Neighborhood'].apply(lambda x: ', '.join(x))

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_neighbourhoods.loc[ (df_neighbourhoods.Neighborhood.isnull() == True) |
               (df_neighbourhoods.Neighborhood == "Not assigned")
               , 'Neighborhood'] = df_neighbourhoods.Borough

df_neighbourhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_neighbourhoods.shape

(103, 3)

Part 2 - Add the latitude and the longitude coordinates of each neighbourhood

install and import library

In [8]:
!conda install -c conda-forge geocoder             #Uncomment this if geocoder is not installed on your system
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


using geocoder with arcgis to get all coordinates by postal code

In [10]:
def getCoordsByPostalCode(postal_code):
    # initialize to None : this variable will allow us to loop until geocoder responds with the coordinates
    lat_lng_coords = None

    # loop until we get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude, longitude

In [11]:
postalCodesWithCoordsList = []

# Loop through each postal code from our dataframe
for postalCode in df_neighbourhoods['PostalCode']:
    # Fill the temp list with the coordinates from geocoder
    latitude, longitude = getCoordsByPostalCode(postalCode)
    postalCodesWithCoordsList.append([postalCode, latitude, longitude])
    
# Transform the temp list into a dataframe
df_coords = pd.DataFrame(postalCodesWithCoordsList)
df_coords.columns = ['PostalCode', 'Latitude', 'Longitude']

# Merge the coordinates dataframe with the original neighbourhoods dataframe (key : postal code)
df_neighbourhoods = pd.merge(df_neighbourhoods, df_coords, on='PostalCode')

df_neighbourhoods_toronto = df_neighbourhoods

df_neighbourhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


Part 3 - Neighbourhoods clustering using venues from FoursquareAPI

install and import library 

In [12]:
!conda install -c conda-forge folium                #Uncomment this if folium is not installed on your system
import folium
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.12.0              |     pyhd8ed1ab_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          64 KB

The following packages will be UPDATED:

  folium                                         0.5.0-py_0 --> 0.12.0-pyhd8ed1ab_0



folium-0.12.0        | 64 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Generate the map of Toronto with the neighbourhoods

Retrieve the coordinates of the city of Toronto so we can draw it using Folium

In [13]:

# initialize to None
lat_lng_coords = None

# loop until we get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('Toronto, Ontario')
    lat_lng_coords = g.latlng

latitude_toronto = lat_lng_coords[0]
longitude_toronto = lat_lng_coords[1]

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


Draw the city of Toronto with the neighbourhoods from the dataframe

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df_neighbourhoods_toronto['Latitude'], df_neighbourhoods_toronto['Longitude'], df_neighbourhoods_toronto['Borough'], df_neighbourhoods_toronto['Neighborhood'], df_neighbourhoods_toronto['PostalCode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

We draw the same map, but with each neighbourhood represented by a circle having a radius equals to the radius size CONST_venuesRadiusScan

In [15]:
map_toronto_radius = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df_neighbourhoods_toronto['Latitude'], df_neighbourhoods_toronto['Longitude'], df_neighbourhoods_toronto['Borough'], df_neighbourhoods_toronto['Neighborhood'], df_neighbourhoods_toronto['PostalCode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.Circle(
        [lat, lng],
        radius=CONST_venuesRadiusScan,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3).add_to(map_toronto_radius) 
    
map_toronto_radius

Get the venues in Toronto, using FoursquareAPI

FoursquareAPI credentials

In [19]:
CLIENT_ID = 'CSN0DC0GVROB3IBVSH5IAWZVZDSHUPDJ0E1GOVYXZLNCVDWI' # your Foursquare ID
CLIENT_SECRET = 'W4GRT4225BPCRT4VTKEXIVOZMPAIHLZZQFI5BUVEDD434C0C' # your Foursquare Secret
VERSION = '20210203' # Foursquare API version

This function will retrieve the FoursquareAPI venues for all the neighbourhoods given in parameters

In [20]:
def getNearbyVenues(postalCodes, boroughs, neighbourhoods, latitudes, longitudes):
    
    venues_list=[]
    # Loop through each neighbourhood given in parameters
    for postalCode, borough, neighbourhood, lat, lng in zip(postalCodes, boroughs, neighbourhoods, latitudes, longitudes):
            
        # create the API request URL to explore the neighbourhood using FoursquareAPI
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            CONST_venuesRadiusScan, 
            CONST_venuesLimit)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue : name, latitude, longitude, and the categories' names
        venues_list.append([(
            postalCode,
            borough,
            neighbourhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # add the venues in the dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                        'PostalCode',
                        'Borough',
                        'Neighborhood', 
                        'Neighborhood Latitude', 
                        'Neighborhood Longitude', 
                        'Venue', 
                        'Venue Latitude', 
                        'Venue Longitude', 
                        'Venue Category'
    ]
    
    return(nearby_venues)

Loop through each Toronto neighbourhood and retrieve its venues using the previous function

In [21]:
# Get the venues for each neighbourhood
toronto_venues = getNearbyVenues(  
                                    postalCodes=df_neighbourhoods_toronto['PostalCode'],
                                    boroughs=df_neighbourhoods_toronto['Borough'],
                                    neighbourhoods=df_neighbourhoods_toronto['Neighborhood'],
                                    latitudes=df_neighbourhoods_toronto['Latitude'],
                                    longitudes=df_neighbourhoods_toronto['Longitude']
                                  )

toronto_venues.head()

,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
2,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
3,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,Tim Hortons,43.802000,-79.198169,Coffee Shop


Analyse each neighborhood : each type of venue will be one hot encoded so we will be able to perform a K-means clustering on the dataframe

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postalCode, borough, and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 
toronto_onehot['Borough'] = toronto_venues['Borough'] 

toronto_onehot.pop('Neighborhood')
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move postalCode, borough, neighborhood column to the first column
for i in range(0, 3):
    toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]

toronto_onehot.head()

,PostalCode,Borough,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by postal code, borough and neighbourhood, using means

In [24]:
toronto_grouped = toronto_onehot.groupby(['PostalCode','Borough', 'Neighborhood']).mean().reset_index()
toronto_grouped.head()

,PostalCode,Borough,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,M1B,Scarborough,"Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.384615
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,M1G,Scarborough,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.047619,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,M1H,Scarborough,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.037037,0.0,0.037037,0.000000


Print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

# Iterate through all the grouped dataframe
for index, row in toronto_grouped.iterrows():
    tempPostalCode = row['PostalCode']
    tempBorough = row['Borough']
    tempNeighborhood = row['Neighborhood']
    
    print("----"+tempPostalCode + " / " + tempBorough + " / " + tempNeighborhood +"----")
    
    # Create a temp df filtered on the current neighbourhood (key : postal code x borouhg x neighbourhood)
    temp = toronto_grouped[
        (toronto_grouped.PostalCode == tempPostalCode) &
        (toronto_grouped.Borough == tempBorough) &
        (toronto_grouped.Neighborhood == tempNeighborhood)
    ].T.reset_index()
    
    temp.columns = ['venue','freq']

    # We skip the key PostalCode x Borough x Neighbourhood : length = 3, iloc[3:]
    temp = temp.iloc[len(CONST_dfColumns):]
    temp['freq'] = temp['freq'].astype(float)
    
    # Round the frequency with two digits
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B / Scarborough / Malvern, Rouge----
                  venue  freq
0           Zoo Exhibit  0.38
1           Coffee Shop  0.15
2  Fast Food Restaurant  0.15
3  Other Great Outdoors  0.08
4                   Spa  0.08


----M1C / Scarborough / Rouge Hill, Port Union, Highland Creek----
                venue  freq
0                Park  0.17
1        Burger Joint  0.17
2      Breakfast Spot  0.17
3  Italian Restaurant  0.17
4          Playground  0.17


----M1E / Scarborough / Guildwood, Morningside, West Hill----
               venue  freq
0               Park  0.07
1  Convenience Store  0.07
2        Pizza Place  0.07
3     Breakfast Spot  0.07
4               Bank  0.04


----M1G / Scarborough / Woburn----
                  venue  freq
0     Indian Restaurant   0.1
1                  Park   0.1
2  Fast Food Restaurant   0.1
3           Coffee Shop   0.1
4           Pizza Place   0.1


----M1H / Scarborough / Cedarbrae----
               venue  freq
0  Indian Restaurant  0.15
1  

                venue  freq
0         Coffee Shop  0.11
1                Bank  0.07
2  Italian Restaurant  0.07
3            Pharmacy  0.07
4    Sushi Restaurant  0.07


----M5P / Central Toronto / Forest Hill North & West, Forest Hill Road Park----
                venue  freq
0               Trail  0.07
1                Café  0.07
2                Bank  0.07
3  Italian Restaurant  0.07
4         Coffee Shop  0.07


----M5R / Central Toronto / The Annex, North Midtown, Yorkville----
                           venue  freq
0             Italian Restaurant  0.05
1                    Coffee Shop  0.05
2                           Café  0.05
3  Vegetarian / Vegan Restaurant  0.04
4                     Restaurant  0.04


----M5S / Downtown Toronto / University of Toronto, Harbord----
                           venue  freq
0                           Café  0.12
1                         Bakery  0.06
2  Vegetarian / Vegan Restaurant  0.05
3                    Coffee Shop  0.03
4                

Put the results into a Pandas dataframe

Function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    # Remove the key PostalCode x Borough x Neighbourhood from the row
    row_categories = row.iloc[len(CONST_dfColumns):]
    
    # Sort ascending
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    # Return the top num_top_venues
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode', 'Borough', 'Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe, and set it with the columns names
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

# add the keys from the grouped dataframe (Postal code x Borough x Neighborhood)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# loop through each rows
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, len(CONST_dfColumns):] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Other Great Outdoors,Spa,Trail,Hobby Shop,Other Repair Shop,Organic Grocery,Optical Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",Park,Burger Joint,Breakfast Spot,Italian Restaurant,Playground,Gym / Fitness Center,Music Venue,Opera House,Movie Theater,Paintball Field
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",Park,Convenience Store,Pizza Place,Breakfast Spot,Bank,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Gymnastics Gym,Filipino Restaurant
3,M1G,Scarborough,Woburn,Indian Restaurant,Park,Fast Food Restaurant,Coffee Shop,Pizza Place,Supermarket,Bank,Bakery,Thrift / Vintage Store,Filipino Restaurant
4,M1H,Scarborough,Cedarbrae,Indian Restaurant,Coffee Shop,Gas Station,Grocery Store,Bakery,Athletics & Sports,Bus Line,Music Store,Caribbean Restaurant,Chinese Restaurant


Cluster neighbourhoods

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('PostalCode', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 2, 2, 2, 2, 2, 2, 0], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [30]:
toronto_merged = df_neighbourhoods_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index(['PostalCode','Borough', 'Neighborhood']), on=['PostalCode','Borough', 'Neighborhood'])

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,1,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Other Great Outdoors,Spa,Trail,Hobby Shop,Other Repair Shop,Organic Grocery,Optical Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,0,Park,Burger Joint,Breakfast Spot,Italian Restaurant,Playground,Gym / Fitness Center,Music Venue,Opera House,Movie Theater,Paintball Field
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470,2,Park,Convenience Store,Pizza Place,Breakfast Spot,Bank,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Gymnastics Gym,Filipino Restaurant
3,M1G,Scarborough,Woburn,43.76812,-79.21761,2,Indian Restaurant,Park,Fast Food Restaurant,Coffee Shop,Pizza Place,Supermarket,Bank,Bakery,Thrift / Vintage Store,Filipino Restaurant
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,2,Indian Restaurant,Coffee Shop,Gas Station,Grocery Store,Bakery,Athletics & Sports,Bus Line,Music Store,Caribbean Restaurant,Chinese Restaurant


Visualise the clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, postalCode, borough, neighborhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(postalCode) + ' - Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

Cluster 0 focuses on Parks, Gyms, and places to practice sports and/or chillout outside.
This cluster is the places that you want to go if you want to do exercise and outdoor activities.

In [33]:
toronto_merged[toronto_merged['Cluster Labels'] == 0].head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,0,Park,Burger Joint,Breakfast Spot,Italian Restaurant,Playground,Gym / Fitness Center,Music Venue,Opera House,Movie Theater,Paintball Field
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.69510,-79.26466,0,Park,Gym Pool,Restaurant,Skating Rink,General Entertainment,Café,Gym,Diner,College Stadium,Thai Restaurant
19,M2K,North York,Bayview Village,43.78112,-79.38060,0,Park,Chinese Restaurant,Bank,Intersection,Trail,Japanese Restaurant,Café,Optical Shop,Pakistani Restaurant,Paintball Field
20,M2L,North York,"York Mills, Silver Hills",43.75698,-79.38060,0,Gym,Park,Intersection,Gym / Fitness Center,Shopping Mall,Furniture / Home Store,Baseball Field,Sandwich Place,Opera House,Other Repair Shop


Cluster 1 focuses on the diversity of coffee shops, Restaurant, and bars.Its 1st common Venue is Zoo Exhibit,
We can see that the downtown of Toronto is highly categorised in this cluster.

In [34]:
toronto_merged[toronto_merged['Cluster Labels'] == 1].head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,1,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Other Great Outdoors,Spa,Trail,Hobby Shop,Other Repair Shop,Organic Grocery,Optical Shop
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.78097,-79.34781,1,Clothing Store,Coffee Shop,Restaurant,Japanese Restaurant,Sandwich Place,Bakery,Bank,Juice Bar,Liquor Store,Fast Food Restaurant
21,M2M,North York,"Willowdale, Newtonbrook",43.79182,-79.41372,1,Coffee Shop,Korean Restaurant,Café,Middle Eastern Restaurant,Restaurant,Grocery Store,Diner,Shopping Mall,Pizza Place,Supermarket
22,M2N,North York,"Willowdale, Willowdale East",43.76774,-79.40728,1,Coffee Shop,Japanese Restaurant,Sandwich Place,Ramen Restaurant,Korean Restaurant,Grocery Store,Bank,Fast Food Restaurant,Middle Eastern Restaurant,Restaurant
23,M2P,North York,York Mills West,43.74778,-79.40033,1,Coffee Shop,Park,Restaurant,Food Court,Bowling Alley,Burrito Place,Business Service,Optical Shop,Gym,Gym / Fitness Center
26,M3B,North York,Don Mills,43.74923,-79.36186,1,Coffee Shop,Pizza Place,Electronics Store,Japanese Restaurant,Supermarket,Restaurant,Bank,Gas Station,Salad Place,Bagel Shop
27,M3C,North York,Don Mills,43.72168,-79.34352,1,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Sandwich Place,Park,Restaurant,Science Museum,Discount Store,Clothing Store
30,M3K,North York,Downsview,43.73384,-79.46828,1,Park,Turkish Restaurant,Coffee Shop,Vietnamese Restaurant,Latin American Restaurant,Thrift / Vintage Store,Sandwich Place,Bakery,Liquor Store,Gas Station
37,M4E,East Toronto,The Beaches,43.67709,-79.29547,1,Pub,Coffee Shop,Beach,Breakfast Spot,Japanese Restaurant,Pizza Place,Grocery Store,Burger Joint,Bakery,Park
38,M4G,East York,Leaside,43.70902,-79.36349,1,Coffee Shop,Bakery,Grocery Store,Indian Restaurant,Bank,Sporting Goods Shop,Furniture / Home Store,Electronics Store,Sushi Restaurant,Sandwich Place


Cluster 2 focuses on the diversity of Restaurant, Grocery storesm, Convenience store.
This cluster is predominant outside of the downtown. 

In [35]:
toronto_merged[toronto_merged['Cluster Labels'] == 2].head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470,2,Park,Convenience Store,Pizza Place,Breakfast Spot,Bank,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Gymnastics Gym,Filipino Restaurant
3,M1G,Scarborough,Woburn,43.76812,-79.21761,2,Indian Restaurant,Park,Fast Food Restaurant,Coffee Shop,Pizza Place,Supermarket,Bank,Bakery,Thrift / Vintage Store,Filipino Restaurant
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,2,Indian Restaurant,Coffee Shop,Gas Station,Grocery Store,Bakery,Athletics & Sports,Bus Line,Music Store,Caribbean Restaurant,Chinese Restaurant
5,M1J,Scarborough,Scarborough Village,43.74446,-79.23117,2,Fast Food Restaurant,Sandwich Place,Ice Cream Shop,Convenience Store,Restaurant,Chinese Restaurant,Coffee Shop,Beer Store,Park,Big Box Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.72582,-79.26461,2,Chinese Restaurant,Grocery Store,Fast Food Restaurant,Coffee Shop,Discount Store,Sandwich Place,Bank,Light Rail Station,Pharmacy,Health Food Store
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.71289,-79.28506,2,Intersection,Grocery Store,Bakery,Sandwich Place,Bus Station,Soccer Field,Gas Station,General Entertainment,Mexican Restaurant,Metro Station
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496,2,Ice Cream Shop,Pharmacy,Sporting Goods Shop,Pizza Place,Park,Gift Shop,Sandwich Place,Coffee Shop,Discount Store,Bank
10,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.75998,-79.26940,2,Indian Restaurant,Coffee Shop,Pizza Place,Furniture / Home Store,Chinese Restaurant,Bank,Fast Food Restaurant,Electronics Store,Restaurant,Park
11,M1R,Scarborough,"Wexford, Maryvale",43.75075,-79.30054,2,Middle Eastern Restaurant,Pizza Place,Asian Restaurant,Grocery Store,Intersection,Furniture / Home Store,Halal Restaurant,Medical Center,Supermarket,Burger Joint
12,M1S,Scarborough,Agincourt,43.79452,-79.26708,2,Chinese Restaurant,Shopping Mall,Cantonese Restaurant,Pizza Place,Department Store,Mediterranean Restaurant,Bakery,Bank,Noodle House,Coffee Shop


Cluster 3 is an outlier, its 1st commont venue is Ski Area. 

In [36]:
toronto_merged[toronto_merged['Cluster Labels'] == 3].head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.83412,-79.21668,3,Ski Area,ATM,Nightclub,Other Repair Shop,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House,Office,Noodle House


Cluster 4 focuses on Food Truck, Park and Baseball field and restaurants.

In [37]:
toronto_merged[toronto_merged['Cluster Labels'] == 4].head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M3M,North York,Downsview,43.73224,-79.50178,4,Food Truck,Park,Baseball Field,Vietnamese Restaurant,ATM,Nightclub,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House
